In [1]:
import numpy as np
import pandas as pd
from queue import PriorityQueue
from itertools import groupby

In [2]:
grid = (
    pd.read_csv("data/20.txt", engine="python", sep=r"\s*", names=range(-1, 1000))
    .dropna(how="all", axis=1)
    .values
)

In [3]:
def get_neighbors(x, y, el="#", op="ne"):
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        if 0 <= x + dx < grid.shape[0] and 0 <= y + dy < grid.shape[1]:
            if op == "ne" and grid[x + dx, y + dy] != el:
                yield x + dx, y + dy
            elif op == "eq" and grid[x + dx, y + dy] == el:
                yield x + dx, y + dy

In [4]:
start = np.where(grid == "S")
start = start[0][0], start[1][0]

goal = np.where(grid == "E")
goal = goal[0][0], goal[1][0]

frontier = PriorityQueue()
frontier.put((0, start))

cost_so_far = {start_state: 0 for _, start_state in frontier.queue}


while not frontier.empty():
    current_cost, current = frontier.get()

    if cost_so_far[current] < current_cost:
        continue

    for next_state in get_neighbors(*current):
        new_cost = current_cost + 1
        if next_state not in cost_so_far or new_cost < cost_so_far[next_state]:
            cost_so_far[next_state] = new_cost
            frontier.put((new_cost, next_state))

predecessors = {}
for state, dist in cost_so_far.items():
    for next_state in get_neighbors(*state):
        if next_state in cost_so_far:
            if dist + 1 == cost_so_far[next_state]:
                if next_state not in predecessors:
                    predecessors[next_state] = []
                predecessors[next_state].append(state)
cheats = set()

stack = [(goal, {goal: 0})]
while stack:
    node, path = stack.pop()
    if node in predecessors:
        for pred in predecessors[node]:
            stack.append((pred, path | {pred: len(path)}))

    for cheat_tile in get_neighbors(*node, op="eq"):
        for jump_tile in get_neighbors(*cheat_tile, op="ne"):
            if jump_tile in path and path[jump_tile] < path[node] - 2:
                cheats.add((path[node] - path[jump_tile] - 2, node, jump_tile))

sum(len(list(g)) for k, g in groupby(sorted(cheats), key=lambda x: x[0]) if k >= 100)

1286

In [5]:
cheats2 = set()

stack = [(goal, {goal: 0})]
while stack:
    node, path = stack.pop()
    if node in predecessors:
        for pred in predecessors[node]:
            stack.append((pred, path | {pred: len(path)}))
    for pred, path_place in path.items():
        path_distance = path[node] - path_place
        cheat_distance = np.abs(pred[0] - node[0]) + np.abs(pred[1] - node[1])
        if cheat_distance <= 20 and cheat_distance < path_distance:
            cheats2.add((path_distance - cheat_distance, node, pred))

sum(len(list(g)) for k, g in groupby(sorted(cheats2), key=lambda x: x[0]) if k >= 100)

989316